# 명탐정 코난 매니아 판별기 (LangGraph)

## OpenAI LLM 준비 및 퀴즈 파일 지정
* 환경 변수(`.env` 파일)에서 API Key 로딩
* 개발 환경에서는 `gpt-4o-mini` 또는 `gpt-3.5-turbo`
* 핵심 실습 환경이라 `gpt-4o` 사용

In [ ]:
import re
import json
import random
import sqlite3
from datetime import datetime
from typing import Literal, TypedDict, Optional, Annotated

import gradio as gr
from dotenv import load_dotenv
from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END

from langchain_teddynote.graphs import visualize_graph

# 경로 및 상수
QUIZ_FILE = "data/quizzes.json"
APPLICANT_FILE = "data/applicants.json"
DB_FILE = "data/quiz_results.db"

QUIZ_COUNT = 3  # 퀴즈 문항
QUIZ_COMMANDS = ["퀴즈", "퀴즈 시작"]


load_dotenv()
llm = ChatOpenAI(model="gpt-4o", temperature=0.5)

## DB 초기화 및 데이터 로딩

In [ ]:
# DB 초기화
def ensure_db():
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS quiz_results (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            taken_at TEXT NOT NULL,
            student_class TEXT,
            student_name TEXT,
            student_id TEXT,
            student_phone TEXT,
            total_score INTEGER,
            total_count INTEGER,
            details_json TEXT
        )
        """
    )
    conn.commit()
    conn.close()

# 퀴즈 로딩 함수
def load_quizzes() -> list[dict]:
    with open(QUIZ_FILE, "r", encoding="utf-8") as f:
        all_q = json.load(f)
    return random.sample(all_q, QUIZ_COUNT)

# 지원자 로딩 함수
def load_applicants() -> list[dict]:
    with open(APPLICANT_FILE, "r", encoding="utf-8") as f:
        return json.load(f)


## 기본 데이터 정의

### 데이터 모델 정의

In [ ]:
# 역할 기반 접근 제어 모델 정의
class RoleRoute(BaseModel):
    """역할 기반 접근 제어를 위한 경로 모델입니다."""
    role: Literal["student", "professor", "unknown"]

# 지원자 정보 모델 정의
class ApplicantInfo(BaseModel):
    """지원자 정보를 담는 클래스입니다."""
    student_class: str = Field(description="지원자의 학급")
    student_name: str = Field(description="지원자의 이름")
    student_id: str = Field(description="지원자의 학번")
    student_phone: str = Field(description="지원자의 전화번호")

# 채점 결과 모델 정의
class GradingResult(BaseModel):
    """단일 문제에 대한 채점 결과를 상세히 담는 클래스입니다."""
    question_id: int = Field(description="문제의 고유 ID")
    question: str = Field(description="채점 대상 문제")
    correct_answer: str = Field(description="문제의 정답")
    user_answer: str = Field(description="사용자가 제출한 답변")
    is_correct: bool = Field(description="정답 여부")
    explanation: str = Field(description="정답에 대한 친절한 해설")

# 퀴즈 채점 결과 모델 정의
class FinalReport(BaseModel):
    """퀴즈의 모든 채점 결과와 최종 점수를 종합한 최종 보고서 클래스입니다."""
    results: list[GradingResult] = Field(description="각 문제별 채점 결과 리스트")
    total_score: str = Field(description="'총점: X/Y' 형식의 최종 점수 요약")

# 보고서 요청 모델 정의
class ReportRequest(BaseModel):
    """최종 보고서 생성을 위한 요청 모델입니다."""
    taken_date: Optional[str] = Field(None, description="YYYY-MM-DD 또는 YYYY.MM.DD")
    student_class: Optional[str] = Field(None, description="반 (예: '2반')")
    report_type: Literal["오답", "성적", "전체"] = "전체"

# LLM 출력 형식 지정
llm_with_role = llm.with_structured_output(RoleRoute)
llm_with_applicant = llm.with_structured_output(ApplicantInfo)
llm_with_report = llm.with_structured_output(FinalReport)


### 상태 정의
그래프(workflow)가 관리할 상태 정보 클래스 `AppState(TypedDict)`

In [ ]:
# 애플리케이션 상태 모델 정의
# 모든 필드를 선택적(total=False)으로 관리 => 상태 관리에 적합한 방식
class AppState(TypedDict, total=False):
    """
    애플리케이션의 전체 상태를 관리하는 중앙 저장소.
    Annotated를 사용하여 각 필드에 대한 설명을 타입 힌트에 포함합니다.
    """
    
    # --- 공통 및 초기 필드 ---
    user_input: Annotated[str, "사용자의 현재 입력값"]
    chat_history: Annotated[list[tuple[str, str]], "UI용 대화 기록 리스트"]
    role: Annotated[Literal["student", "professor", "unknown"], "현재 사용자의 역할"]

    # --- 응시자(student) 흐름 관련 필드 ---
    applicant: Annotated[ApplicantInfo, "응시자 정보"]
    questions: Annotated[list[dict], "생성된 퀴즈 질문 목록"]
    quiz_index: Annotated[int, "현재 진행 중인 퀴즈의 인덱스"]
    user_answers: Annotated[list[str], "사용자가 제출한 답변 목록"]
    grading_input_str: Annotated[str, "채점을 위해 LLM에 전달할 종합 문자열"]
    final_report: Annotated[FinalReport, "최종 채점 결과 보고서"]

    # --- 교수(professor) 리포트 흐름 관련 필드 ---
    report_req: Annotated[ReportRequest, "교수가 요청한 리포트 상세 정보"]
    report_output_md: Annotated[str, "생성된 리포트의 마크다운 결과물"]


## Agent 노드 함수 구현

**일반적인 함수** 이름은 **동사** 로 시작하고, **노드 함수** 인 경우 **역할(행위자)** 을 명시해서 작명 권고

### 1.1. LLM에 의한 역할 분류 함수

In [ ]:
def classify_role(text: str) -> Literal["student", "professor", "unknown"]:
    """ 사용자 입력을 분석하여 역할을 분류하는 함수입니다."""

    system_message = """  
    당신은 사용자 유형을 분류하는 매우 정확한 라우터입니다. 사용자의 입력을 보고 'student', 'professor', 'unknown' 중 하나로 분류해주세요.

    ## 분류 기준:
    1. 'student': 반, 이름, 학번 등 개인정보를 포함하여 퀴즈 응시를 시도하는 경우.
    2. 'professor': 날짜, 반, '리포트' 또는 '성적'과 같은 키워드를 포함하여 결과를 조회하려는 경우.
    3. 'unknown': 위 두 경우에 해당하지 않는 모든 애매한 경우.

    ## 예시:
    - 입력: "1반 홍길동 S25B001 010-1111-2222", 분류: 'student'
    - 입력: "2025-07-07 2반 성적 순위 리포트 좀 보여줘", 분류: 'professor'
    - 입력: "안녕하세요", 분류: 'unknown'
    - 입력: "퀴즈를 풀고 싶어요.", 분류: 'unknown' (퀴즈 응시를 원하지만, 식별 정보가 없으므로 'unknown' 처리 후 안내)

    ## 출력 형식:
        JSON {"role": "student|professor|unknown"} 한 값만 주세요.
    """

    prompt = ChatPromptTemplate.from_messages([
        ( "system", system_message.strip()),
        ("human", "{input_text}")
    ])

    try:
        response = (prompt | llm_with_role).invoke({"input_text": text})
        return response.role
    except Exception:
        return "unknown"


### 1.2. 역할에 따른 엔트리 판단 노드

※ 엔트리 노드는 그래프 빌드 직전에 선언하여 선행 노드 참조할 수 있도록 구성

In [ ]:
def entry_router(state: AppState) -> str:
    """역할 분류 및 진입점 라우터 노드 함수입니다."""

    role = state.get("role")
    if not role:
        user_input = state.get("user_input")
        if user_input:
            role = classify_role(user_input)
        else:
            role = "unknown"

    if role == "student":
        return "student_entry"
    elif role == "professor":
        return "professor_entry"
    else:
        return "unknown_entry"

### 1.3. 엔트리 판단을 위한 도움말 제공 노드

In [ ]:
def entry_helper(state: AppState) -> AppState:
    """알 수 없는 역할에 대한 도움말 메시지를 출력하는 노드 함수입니다."""

    state["chat_history"].append((
        "assistant",
        "학생은 '1반 김영희 S25B002 010-0000-0000' 처럼 본인 정보를 입력하세요.\n"
        "교수는 '2025-07-07 2반 리포트 출력'처럼 날짜와 반을 포함해 입력하세요."
    ))
    return state

### 2.1. LLM에 의한 응시자 정보 추출 : 입력된 응시자 정보를 JSON 형식으로 전달

In [ ]:
def parse_applicant_info(text: str) -> ApplicantInfo | None:
    """사용자 입력에서 지원자 정보를 추출하는 함수입니다."""

    system_message = """  
    아래 문장에서 반(student_class), 이름(student_name), 학번(student_id), 전화번호(student_phone)을 추출하세요.
    - 반: 숫자와 '반'이 포함된 문자열 (예: '1반', '2반') 
    - 이름: 한글로 된 이름
    - 학번: 'S'로 시작하는 영문자와 숫자의 조합    
    - 전화번호: 하이픈(-)이 포함될 수 있는 8개 이상의 숫자 형식

    ## 예시:
    - 입력: "1반 홍길동 S25B001 010-1111-2222"
    - 출력: {"student_class": "1반", "student_name": "홍길동", "student_id": "S25B001", "student_phone": "010-1111-2222"}
    """

    prompt = ChatPromptTemplate.from_messages([
        ( "system", system_message.strip()),
        ("human", "{input_text}")
    ])

    try:
        response = (prompt | llm.with_applicant).invoke({"input_text": text})
        if not response.student_name or not response.student_id:
            return None
        return response
    except Exception:
        return None


### 2.2. 응시자 정보 검증 노드

In [ ]:

def applicant_validator(state: AppState) -> AppState:
    """추출한 응시자 정보로 등록된 사용자 여부 및 퀴즈 응시 여부를 확인하는 노드 함수입니다."""
    
    user_input = state.get("user_input", "")
    applicant = parse_applicant_info(user_input)
    if not applicant:
        state["chat_history"].append(("assistant", "응시자 정보를 인식하지 못했습니다. 예) 1반 김영희 S25B002 010-0000-0000"))
        return state

    # 등록된 응시자 확인
    try:
        roster = load_applicants()
    except Exception:
        roster = []

    exists = next((r for r in roster if r.get("student_id") == applicant.student_id), None)
    if not exists:
        state["chat_history"].append(("assistant", f"등록된 응시자를 찾지 못했습니다: {applicant.student_id}"))
        return state

    # 이미 응시했는지 확인
    conn = sqlite3.connect(DB_FILE)
    cur = conn.cursor()
    cur.execute("SELECT taken_at,total_score FROM quiz_results WHERE student_id=? ORDER BY id DESC LIMIT 1", (applicant.student_id))
    row = cur.fetchone()
    conn.close()
    if row:
        taken_at, total_score = row
        state["chat_history"].append(("assistant", f"이미 응시 기록이 있습니다. 응시일자: {taken_at}, 점수: {total_score}"))
        return state

    # 응시자 검증 통과
    state["applicant"] = applicant
    message = f"{applicant.student_class} {applicant.student_name}님, 퀴즈를 시작하려면 '퀴즈 시작'이라고 입력하세요."
    state["chat_history"].append(("assistant", message))
    return state


### 2.3. 응시자 퀴즈 출제 노드

In [ ]:
def quiz_setter(state: AppState) -> AppState:
    """퀴즈 문항을 설정하는 노드 함수입니다."""
    
    questions = load_quizzes()
    if not questions:
        state["chat_history"].append(
            ("assistant", "퀴즈를 불러오는 데 실패했거나 풀 수 있는 문제가 없습니다.")
        )
        state["questions"] = []
        return state

    state["questions"] = questions
    state["quiz_index"] = 0
    state["user_answers"] = []
    state["final_report"] = None
    state["chat_history"].append(("assistant", f"퀴즈를 시작합니다. 총 {len(questions)}문항입니다."))
    return state

### 2.4. 퀴즈 진행 여부 판단 조건부 노드

In [ ]:
def continue_quiz_condition(state: AppState) -> str:
    """퀴즈 진행 여부를 판단하는 조건부 노드 함수입니다."""
    
    if not state.get("questions"):
        return "quiz_setter"
    
    if state["quiz_index"] < len(state["questions"]):
        return "quiz_popper" 
    else:
        return "quiz_grader"

### 2.5. 개별 퀴즈 출력 노드

In [ ]:
def quiz_popper(state: AppState) -> AppState:
    """개별 퀴즈 문항을 출력하는 노드 함수입니다."""

    if not state.get("questions"):
        return state
    
    qi = state.get("quiz_index", 0)
    if qi < len(state["questions"]):
        q = state["questions"][qi]
        text = f"문제 {qi + 1}: {q['question']}"
        if q["type"] == "multiple_choice":
            choices = "\n".join([f"{i + 1}. {c}" for i, c in enumerate(q["choices"])])
            text += "\n" + choices
        state["chat_history"].append(("assistant", text))
    return state


### 개별 퀴즈 답변 저장 노드

In [ ]:
def answer_collector(state: AppState) -> AppState:
    """개별 퀴즈 답변을 수집하는 노드 함수입니다."""

    if not state.get("questions"):
        return state
    
    qi = state.get("quiz_index", 0)
    if qi >= len(state["questions"]):
        return state

    q = state["questions"][qi]
    user_ans = state["user_input"].strip()
    if not user_ans:
        state["chat_history"].append(("assistant", "답변을 입력해 주세요."))
        return state

    processed = user_ans
    if q["type"] == "multiple_choice":
        try:
            idx = int(user_ans) - 1
            if 0 <= idx < len(q["choices"]):
                processed = q["choices"][idx]
        except Exception:
            pass
    state["user_answers"].append(processed)
    state["quiz_index"] = qi + 1
    return state



### 2.6. 퀴즈 채점 요청 노드 

In [ ]:
def quiz_prepare_grading(state: AppState) -> AppState:
    """퀴즈 채점을 위해 입력 데이터를 준비하는 노드 함수입니다."""
    
    parts = ["채점 대상 데이터"]
    for q, a in zip(state["questions"], state["user_answers"]):
        parts.append(f"\n---\nid: {q['id']}\n문제: {q['question']}")
        if q["type"] == "multiple_choice":
            parts.append(f"선택지: {', '.join(q['choices'])}")
        parts.append(f"정답: {q['answer']}")
        parts.append(f"사용자 답변: {a}")
    state["grading_input_str"] = "\n".join(parts)
    state["chat_history"].append(("assistant", "채점을 진행합니다..."))
    return state


### 퀴즈 시작(상태 초기화) 노드
* 상태 갱신의 목적: "퀴즈 시작"이라는 트리거에서 호출되고 모든 상태 정보를 초기화
    - 문제 리스트 세팅: `state["questions"]`
    - 진행 인덱스 초기화: `state["quiz_index"]`
    - 답변 내역, 리포트 등 모두 초기화
    - 유저(`user`)/AI(`assistant`) 대화 이력(`chat_history`)에 안내 메시지 추가

* `questions = load_quiz()` : 랜덤하게 생성된 QUIZ_COUNT로 개수의 퀴즈 문항
* `state["chat_history"].append()` : UI 채팅창에 출력할 정보 추가

In [ ]:
def start_quiz(state: QuizState) -> QuizState:
    """퀴즈를 시작하고 상태를 초기화합니다."""
    questions = load_quiz()
    if not questions:
        state["chat_history"].append(
            ("assistant", "퀴즈를 불러오는 데 실패했거나 풀 수 있는 문제가 없습니다.")
        )
        state["questions"] = []
        return state

    state["questions"] = questions
    state["quiz_index"] = 0
    state["user_answers"] = []
    state["final_report"] = None
    state["chat_history"].append(("assistant", "명탐정 코난 퀴즈를 시작합니다!"))
    return state

### 문제 출제 노드
* 상태(state) 갱신의 목적 : 문제를 사용자에게 출력(`chat_history`에 포맷팅된 문제 내용을 assistant 메시지로 추가)
    - `quiz_index`에 해당하는 문제(`q`)를 가져와 포맷(문자열 변환)
    - 문제 타입이 `multiple_choice`면, 선택지도 보기 형태로 문자열로 추가
    - `chat_history`에 ("assistant", 문제 텍스트) 튜플을 append

In [ ]:
def ask_question(state: QuizState) -> QuizState:
    """현재 quiz_index에 맞는 문제를 포맷하여 chat_history에 추가합니다."""
    idx = state["quiz_index"]
    q = state["questions"][idx]

    text = f"문제 {idx + 1}: {q['question']}"
    if q["type"] == "multiple_choice":
        choices = [f"{i + 1}. {c}" for i, c in enumerate(q["choices"])]
        text += "\n" + "\n".join(choices)

    state["chat_history"].append(("assistant", text))
    return state

### 사용자 답변 처리 노드
* 상태(state) 갱신의 목적 : 사용자의 답변(`user_input`)을 처리
    - 현재 문제(`quiz_index`)에 맞는 답변을 파싱, 전처리, 정제
    - 선택형 문제(`multiple_choice`)라면 사용자가 번호로 입력했을 때 실제 보기 텍스트로 변환
    - 가공/정제된 답변(`processed_answer`)을 `user_answers` 리스트에 append
    - 진행 인덱스(`quiz_index`)를 +1 하여 다음 문제로 이동할 준비

* `user_input = state["user_input"].strip()` : `state["user_input"]`은 `chat_fn` 에서 입력됨. `strip()`은 앞뒤 공백 모두 제거
* `except (ValueError, IndexError): pass` : 사용자가 "a"처럼 숫자로 변환할 수 없는 값을 입력하거나(ValueError) "5"처럼 선택지 범위를 벗어나는 숫자를 입력하면(IndexError), pass 즉 아무런 작업도 하지 않고 그냥 넘어가서, 사용자가 입력한 값이 `state["user_answers"]`로 저장됨.

In [ ]:
def process_and_store_answer(state: QuizState) -> QuizState:
    """사용자 답변을 처리하고 저장한 뒤, 다음 문제로 넘어갑니다."""
    idx = state["quiz_index"]
    q = state["questions"][idx]
    user_input = state["user_input"].strip()

    # 빈 입력일 경우 안내 메시지만 추가하고, 인덱스는 그대로 유지
    if not user_input:
        state["chat_history"].append(
            ("assistant", "답변을 입력해 주세요."))
        return state

    processed_answer = user_input
    if q["type"] == "multiple_choice":
        try:
            sel = int(user_input) - 1
            if 0 <= sel < len(q["choices"]):
                processed_answer = q["choices"][sel]
        except (ValueError, IndexError):
            pass

    state["user_answers"].append(processed_answer)
    state["quiz_index"] += 1
    return state

### 채점을 위해 LLM에 전달할 프롬프트 생성 노드
* 상태(state) 갱신의 목적 : 
    - 사용자에게 "채점 중" 안내를 위해 `chat_history`에 추가
    - 채점 대상 데이터를 `state["grading_input_str"]`에 저장 → 다음 노드(LLM 채점 호출 등)에서 활용
* `zip(state["questions"], state["user_answers"])` : 각 문제와 각 답변을 쌍으로 묶어서 반환 (튜플: (문제, 답변))
* `enumerate(...)` : 튜플 쌍에 인덱스 번호(`i`)를 붙여서 (`i, (q, a)`) 형태로 반환

In [ ]:
def prepare_grading_prompt(state: QuizState) -> QuizState:
    """채점을 위해 LLM에 전달할 프롬프트를 생성합니다."""
    state["chat_history"].append(
        ("assistant", "채점을 진행합니다..."))
        
    parts = [
        "지금부터 아래의 문제와 정답, 그리고 사용자의 답변을 보고 채점을 시작해주세요."
    ]
    for i, (q, a) in enumerate(zip(state["questions"], state["user_answers"])):
        parts.append(f"\n--- 문제 {i + 1} ---")
        parts.append(f"문제: {q['question']}")
        if q["type"] == "multiple_choice":
            parts.append(f"선택지: {', '.join(q['choices'])}")
        parts.append(f"정답: {q['answer']}")
        parts.append(f"사용자 답변: {a}")

    state["grading_input_str"] = "\n".join(parts)
    return state

### LLM 채점 및 파싱 노드
* 상태(state) 갱신의 목적 : 
    - 채점 대상 데이터를 `state["grading_input_str"]`를 LLM에 전달
    - LLM의 채점 결과를 FinalReport 클래스 JSON 데이터로 수신하여 `state["final_report"]`에 저장
    
- `chain = prompt | llm_with_final_report` : 프롬프트와 FinalReport를 출력으로 하는 LLM을 LCEL 체인으로 연결



In [ ]:
def grade_with_llm_and_parse(state: QuizState) -> QuizState:
    """LLM을 호출하여 채점하고 결과를 파싱합니다."""
    system_message = """
    당신은 '명탐정 코난' 퀴즈의 전문 채점관입니다. 주어진 문제, 정답, 사용자 답변을 바탕으로 채점해주세요. 
    각 문제에 대해 정답 여부를 판단하고 친절한 해설을 덧붙여주세요. 
    모든 채점이 끝나면, 마지막에는 '총점: X/Y' 형식으로 최종 점수를 반드시 요약해서 보여줘야 합니다. 
    반드시 지정된 JSON 형식으로만 답변해야 합니다."""

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_message),
            ("human", "{grading_data}"),
        ]
    )

    try:
        # 체인 생성
        chain = prompt | llm_with_final_report
        report = chain.invoke({"grading_data": state["grading_input_str"]})
        state["final_report"] = report
    except Exception as e:
        print(f"채점 중 오류 발생: {e}")
        error_report = FinalReport(results=[], total_score="채점 오류가 발생했습니다.")
        state["final_report"] = error_report

    return state

### 최종 리포트(FinalReport)를 사용자에게 보여줄 문자열로 변환 노드
상태(state) 갱신의 목적 : LLM이 제공한 최종 채점 결과를 문제별로 정오, 정답, 제출 답변, 해설을 보기 좋게 포매팅
- 마지막에 총점도 포함
- 안내 메시지(재시작 방법 등)도 추가
- 결과를 `assistant` 메시지로 `chat_history`에 append

워크플로우 마지막 단계로 구조화된 채점 결과 객체(`final_report_obj`)를 사람이 읽을 수 있는 문자열로 변환하여 `chat_history`에 추가함으로써 사용자에게 "마지막 답변"을 제공

In [ ]:
def format_final_report(state: QuizState) -> QuizState:
    """파싱된 최종 리포트를 사용자에게 보여줄 문자열로 변환합니다."""
    final_report_obj = state["final_report"]
    report_parts = ["채점이 완료되었습니다! 🎉\n"]

    if final_report_obj and final_report_obj.results:
        for i, res in enumerate(final_report_obj.results):
            is_correct_text = "✅ 정답" if res.is_correct else "❌ 오답"
            report_parts.append(f"--- 문제 {i + 1} ---")
            report_parts.append(f"문제: {res.question}")
            report_parts.append(f"정답: {res.correct_answer}")
            report_parts.append(f"제출한 답변: {res.user_answer}")
            report_parts.append(f"결과: {is_correct_text}")
            report_parts.append(f"해설: {res.explanation}\n")
        report_parts.append(f"**{final_report_obj.total_score}**")
    else:
        report_parts.append("채점 결과를 생성하는 데 실패했습니다.")

    report_parts.append("\n퀴즈를 다시 시작하려면 '퀴즈 시작'이라고 입력해주세요.")
    state["chat_history"].append(("assistant", "\n".join(report_parts)))
    return state

### 퀴즈 시작 명령어 예외 처리 노드
사용자가 “퀴즈 시작” 명령어를 입력하지 않은 경우, 올바른 명령어 사용법(트리거 문구)을 안내하는 메시지를 `chat_history`에 `assistant` 역할로 추가

In [ ]:
def handle_invalid_start(state: QuizState) -> QuizState:
    """퀴즈 시작 명령어가 아닐 경우 안내 메시지를 추가합니다."""
    help_message = "'퀴즈' 또는 '퀴즈 시작'이라고 입력하면 퀴즈가 시작됩니다."
    state["chat_history"].append(("assistant", help_message))
    return state

### StateGraph 조건부 함수
분기(`Edge Condition`) 함수로, 입력(state)을 분석해서 다음 실행할 노드 이름을 반환
- `should_continue_quiz` : 퀴즈가 아직 남아 있는지, 아니면 채점 단계로 넘어가야 하는지 다음 노드 이름을 결
- `route_initial_input` : 사용자의 첫 입력이 이미 퀴즈를 진행 중인 상태인지, “퀴즈 시작” 명령어인지, 또는 잘못된 입력인지 구분하여 시작 노드를 결정

In [ ]:
def should_continue_quiz(state: QuizState) -> str:
    """퀴즈를 계속할지, 채점을 시작할지 결정합니다."""
    if state["quiz_index"] < len(state["questions"]):
        return "continue_quiz"
    else:
        return "grade_quiz"


def route_initial_input(state: QuizState) -> str:
    """사용자의 입력을 분석하여 워크플로우의 시작점을 결정합니다."""
    if state.get("questions") and state["questions"]:
        return "process_answer"
    else:
        if state["user_input"].strip().lower() in QUIZ_COMMANDS:
            return "start_quiz"
        else:
            return "invalid_start"

## StateGraph 정의 및 컴파일

### 노드 추가
- 각 노드는 하나의 “기능적 단계”를 담당 : `start_quiz`는 상태 초기화, `ask_question`은 문제 출제, `process_answer`는 답변 처리 등
- `invalid_start`는 잘못된 시작 입력 안내

### 조건부 진입점 설정
- 워크플로우의 초기 입력을 분석하여 “퀴즈 시작” 명령이면 `start_quiz`, 이미 진행 중이면 `process_answer`, 그 외엔 `invalid_start`로 각기 다른 노드에서 실행이 시작됨

### 노드 연결(엣지 설정)

직접 연결:
- `start_quiz` → `ask_question`: 퀴즈 시작 후 문제 출제
- `prepare_grading` → `grade_and_parse` → `format_report`: 채점 준비→실행→리포트 출력
- `format_report` → `END`: 채점 결과 출력 후 종료

조건부 연결:
- `process_answer` 다음에 `should_continue_quiz`로 검사, 문제 남아 있으면 `ask_question`, 다 풀었으면 `prepare_grading`
- 노드 종료 연결 : `ask_question` → `END`, `invalid_start` → `END` (`ask_question → END`는 “한 문제만 내는” 임시 흐름처럼 보이니, 여러 문제 반복엔 반드시 위 조건부 연결이 필요!)

In [ ]:
workflow = StateGraph(QuizState)

# 노드 추가
workflow.add_node("start_quiz", start_quiz)
workflow.add_node("ask_question", ask_question)
workflow.add_node("process_answer", process_and_store_answer)
workflow.add_node("prepare_grading", prepare_grading_prompt)
workflow.add_node("grade_and_parse", grade_with_llm_and_parse)
workflow.add_node("format_report", format_final_report)
workflow.add_node("invalid_start", handle_invalid_start)

# 조건부 진입점 설정
workflow.set_conditional_entry_point(
    route_initial_input,
    {
        "start_quiz": "start_quiz",
        "process_answer": "process_answer",
        "invalid_start": "invalid_start",
    },
)

# 엣지 연결
workflow.add_edge("start_quiz", "ask_question")
workflow.add_edge("ask_question", END)
workflow.add_edge("invalid_start", END)

workflow.add_conditional_edges(
    "process_answer",
    should_continue_quiz,
    {"continue_quiz": "ask_question", "grade_quiz": "prepare_grading"},
)
workflow.add_edge("prepare_grading", "grade_and_parse")
workflow.add_edge("grade_and_parse", "format_report")
workflow.add_edge("format_report", END)

quiz_app = workflow.compile()

## 그래프 시각화

In [ ]:
visualize_graph(quiz_app)

## UI 인터페이스 함수
`init_state()` :
- 앱을 초기화할 때 사용할 빈 상태(딕셔너리) 반환
- `quiz_state` 아래에 문제/대화 이력만 먼저 준비
- 다른 필드는 워크플로우 도중 필요에 따라 추가됨(예: quiz_index, user_answers, final_report 등)

`chat_fn(user_input, state)` : Gradio 챗봇 UI에서 실제로 호출되는 메인 처리 함수


In [ ]:
def init_state():
    return {"quiz_state": {"questions": [], "chat_history": []}}


def chat_fn(user_input, state):
    quiz_state = state["quiz_state"]

    if quiz_state.get("final_report") and user_input.strip().lower() in QUIZ_COMMANDS:
        quiz_state = init_state()["quiz_state"]

    current_chat_history = quiz_state.get("chat_history", [])
    current_chat_history.append(("user", user_input))

    graph_input = {
        **quiz_state,
        "user_input": user_input,
        "chat_history": current_chat_history,
    }

    new_state = quiz_app.invoke(graph_input)

    state["quiz_state"] = new_state

    chat_display = [
        {"role": role, "content": content}
        for role, content in new_state["chat_history"]
    ]

    return chat_display, state

## Gradio UI 정의
Gradio의 Blocks UI 구성 방식을 사용해 명탐정 코난 퀴즈 챗봇 인터페이스를 구성

In [ ]:
# Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("### 🕵️ 명탐정 코난 매니아 판별기 (by LangGraph)")

    chatbot = gr.Chatbot(
        label="명탐정 코난 퀴즈 챗봇",
        height=400,
        avatar_images=("data/avatar_user.png", "data/avatar_conan.png"),
        type="messages",
    )

    txt = gr.Textbox(placeholder="'퀴즈 시작'을 입력해보세요!", show_label=False)
    state = gr.State(init_state())

    txt.submit(chat_fn, inputs=[txt, state], outputs=[chatbot, state])
    txt.submit(lambda: "", None, txt)

    demo.launch()

-----
** End of Documents **